In [ ]:
## Data pre-process

### Convert XML dataset to CSV

In [5]:
from xml.etree import ElementTree as ET
import pandas as pd
import csv


# Dataset: clinical signs and symptoms in rare diseases
tree = ET.parse('data/en_product4.xml')
root = tree.getroot()


headers = ['HPODisorderSetStatus_id', 'Disorder_id', 'OrphaCode', 'ExpertLink', 'Name', 'DisorderType_id',
            'DisorderType_name', 'DisorderGroup_id', 'DisorderGroup_Name', 'HPODisorderAssociation_id',
            'HPO_id', 'HPOId', 'HPOTerm', 'HPOFrequency_id', 'HPOFrequency_Name', 'DiagnosticCriteria_id',
            'DiagnosticCriteria_Name', 'Source', 'ValidationStatus', 'Online', 'ValidationDate']


def find_value(row_data, source_tag, target_tag_name, field, text=True):
    """Finds a sub-tag of a source tag and inputs its value into a dictionary containing the current row's data
    
    Args:
        row_data (dict):
            the data for the current row associated with the csv fields
        source_tag (Element):
            XML parent tag to search from
        target_tag_name (str):
            Name of the sub-tag to find
        field (str):
            Field in the csv file
        text (bool):
            Indicates if the value of the tag to retrieve is its inner text or its id attribute
    """
    tag = source_tag.find(target_tag_name)
    tag_v = ''
    
    if tag is not None:  #retrieving either the inner text or the id attribute of the tag
        if text: tag_v = tag.text
        elif (len(tag.attrib) > 0): tag_v = tag.attrib['id']
    row_data[field] = tag_v if tag_v is not None else ''
    
    return tag


with open('data/en_product4.csv', 'w', encoding='utf-8') as fd:
    csvwriter = csv.DictWriter(fd, delimiter=',', fieldnames=headers)
    csvwriter.writeheader()
    
    # iterating through all the disorders
    for status in root.find('HPODisorderSetStatusList').findall('HPODisorderSetStatus'):
        row_data = {}
        row_data['HPODisorderSetStatus_id'] = status.attrib['id']
        
        disorder_tag = find_value(row_data, status, 'Disorder', 'Disorder_id', text=False)
        find_value(row_data, disorder_tag, 'OrphaCode', 'OrphaCode', text=True)
        find_value(row_data, disorder_tag, 'ExpertLink', 'ExpertLink', text=True)
        find_value(row_data, disorder_tag, 'Name', 'Name', text=True)
        
        disordertype_tag = find_value(row_data, disorder_tag, 'DisorderType', 'DisorderType_id', text=False)
        find_value(row_data, disordertype_tag, 'Name', 'DisorderType_name', text=True)
        disordergroup_tag = find_value(row_data, disorder_tag, 'DisorderGroup', 'DisorderGroup_id', text=False)
        find_value(row_data, disordergroup_tag, 'Name', 'DisorderGroup_Name', text=True)
        
        for field in ['Source', 'ValidationStatus', 'Online', 'ValidationDate']:
            find_value(row_data, status, field, field, text=True)
        
        # iterating through all the disorder associations and writing a row for each
        for association in disorder_tag.find('HPODisorderAssociationList').findall('HPODisorderAssociation'):
            row_data['HPODisorderAssociation_id'] = association.attrib['id']
            
            hpo_tag = find_value(row_data, association, 'HPO', 'HPO_id', text=False)
            find_value(row_data, hpo_tag, 'HPOId', 'HPOId', text=True)
            find_value(row_data, hpo_tag, 'HPOTerm', 'HPOTerm', text=True)
            hpofrequency_tag = find_value(row_data, association, 'HPOFrequency', 'HPOFrequency_id', text=False)
            find_value(row_data, hpofrequency_tag, 'Name', 'HPOFrequency_Name', text=True)
            
            diagnosticcriteria_tag = find_value(row_data, association, 'DiagnosticCriteria', 'DiagnosticCriteria_id', text=False)
            find_value(row_data, diagnosticcriteria_tag, 'Name', 'DiagnosticCriteria_Name', text=True)
            
            csvwriter.writerow(row_data)

In [7]:
import pandas as pd

df = pd.read_csv('data/en_product4.csv')
print(df.shape)
print(df[df["DiagnosticCriteria_Name"].notnull()])

(112241, 21)
        HPODisorderSetStatus_id  Disorder_id  OrphaCode  \
789                          51          109        558   
790                          51          109        558   
791                          51          109        558   
792                          51          109        558   
794                          51          109        558   
...                         ...          ...        ...   
111855                     4784        17185     158057   
111856                     4784        17185     158057   
111857                     4784        17185     158057   
111858                     4784        17185     158057   
112237                     4807        18531     206599   

                                               ExpertLink  \
789     http://www.orpha.net/consor/cgi-bin/OC_Exp.php...   
790     http://www.orpha.net/consor/cgi-bin/OC_Exp.php...   
791     http://www.orpha.net/consor/cgi-bin/OC_Exp.php...   
792     http://www.orpha.net/conso

### Merge ORDO, HP and HOOM ontologies

https://bioportal.bioontology.org/ontologies/ORDO?p=summary
https://bioportal.bioontology.org/ontologies/HP?p=summary
https://bioportal.bioontology.org/ontologies/HOOM?p=summary

Using Protégé, merge HP into HOOM and ORDO into HOOM
'HOOM is a module that qualifies the annotation between a clinical entity and phenotypic abnormalities according to a frequency and by integrating the notion of diagnostic criterion.'